# Capstone Coursera-IBM Python for Data-Science

This notebook will be used to be peer-reviewed in the module 3 of the capstone IBM.

## Module 3

Import some libraries

In [28]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Part 1

#### Get data from Wikipedia

In [29]:
data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Use BeautifulSoup to parse data

In [30]:
data = BeautifulSoup(data, 'html.parser')
zc=[] #zipcode
bl=[] #borough list
nl=[] #neighborhood

In [31]:
for r in data.find('table').find_all('tr'):
    c=r.find_all('td')
    if(len(c)>0):
        #print(c[0].text)
        zc.append(c[0].text.rstrip('\n'))
        bl.append(c[1].text.rstrip('\n'))
        nl.append(c[2].text.rstrip('\n'))

In [32]:
t_df=pd.DataFrame({'PostalCode':zc,
                   'Borough':bl,
                   'Neighborhood':nl})

In [33]:
t_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [34]:
t_df.PostalCode.duplicated().any()

False

There's no duplicate value in PostalCode.

#### Drop Borough='Not assigned' lines and correct Neighborhood='Not assigned'

In [43]:
t_df=t_df[t_df.Borough!="Not assigned"].reset_index(drop=True)
for index, r in t_df.iterrows():
    if r["Neighborhood"] == "Not assigned":
        r["Neighborhood"] = r["Borough"]

In [44]:
t_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


It seems that something changes on wikipedia as the neiborhood seems to be separed by a ' / '

#### Replace '/' by ','

In [45]:
for r in t_df.index:
    t_df.Neighborhood[r]=t_df.Neighborhood[r].replace(' / ',', ')

In [46]:
t_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Finally, last question of part 1 : shape of the DataFrame

In [16]:
t_df.shape

(103, 3)

### Part 2

#### Load the coordinates from the CSV given by Coursera

In [17]:
coord=pd.read_csv("Geospatial_Coordinates.csv")
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
coord['Postal Code'].duplicated().any()

False

There's no duplicate value in Postal Code.
The column is entitled "PostalCode" in t_df, the Toronto DataFrame from Wikipedia and "Postal Code" in coord, the DataFrame form Coursera. It should be made consistent before to merge.

In [22]:
coord.rename(columns={"Postal Code":"PostalCode"},inplace=True)
coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the two tables

In [24]:
t_df_full=t_df.merge(coord, on="PostalCode", how="left")

In [25]:
t_df_full.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [27]:
t_df_full

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
